In [8]:
import pandas as pd
from scipy import stats
import numpy as np

In [9]:
tasks = ['binary', 'type']
representations = ['bow', 'freq', 'tfidf']
models = ['decision-tree', 'svm', 'naive-bayes']

In [10]:
#find the mode of a numpy array
def mode(array):
    freq = np.bincount(array)
    return np.argmax(freq)

In [21]:
#combine columns in a dataframe whose names contain certain substrings
def combine_columns(df, substrings):
    '''
    df: the input dataframe that has many column names that contain certain common substrings
    substrings: the list of substrings used for combination
    
    returns: `output_df`, a DataFrame with `substrings` as its columns. Each row is the mode
    of the value for that row of all of the columns in `df` that contained that substring.
    '''
    output_df = pd.DataFrame()
    
    for column_substring in substrings:
        temp_df = pd.DataFrame()
        to_be_combined = [column for column in df.columns.values if column_substring in column]
        
        for column in to_be_combined:
            temp_df[column] = df[column]
            
        array = np.array(temp_df).astype(int)
        array = [mode(row) for row in array]
            
        output_df[column_substring] = array
    
    return output_df

In [41]:
#find which models performed best using basic accuracy score
def evaluate_models(df, substrings):
    output_df = pd.DataFrame()
    temp_df = combine_columns(df, substrings)
    for column in substrings:
        temp_df[column] = temp_df[column] == df['Actual']
        output_df[column] = [sum(temp_df[column])/len(temp_df),]
    output_df.index = ['Score']
    return output_df

#test model significance difference w/ Friedman Chi^2 while combining certain columns
def test_model_difference(df, substrings):
    df = combine_columns(df, substrings)
    friedman_result = stats.friedmanchisquare(
        df[substrings[0]],
        df[substrings[1]],
        df[substrings[2]],
        )
    return friedman_result

In [42]:
for task in tasks:
    df = pd.read_csv(f'{task}_predictions.csv')
    temp_df = evaluate_models(df, df.columns.values).transpose()
    temp_df = temp_df.sort_values('Score', ascending=False)
    friedman_result = test_model_difference(df, df.columns.values[1:])
    print(f'Task: {task}.')
    print(f'Friedman chi square test p-value: {friedman_result.pvalue}')
    print(temp_df)
    print()

Task: binary.
Friedman chi square test p-value: 6.302286079316834e-89
                                       Score
Actual                              1.000000
svm-bow-classification              0.769672
decision-tree-tfidf-classification  0.760656
svm-freq-classification             0.757377
decision-tree-freq-classification   0.756557
svm-tfidf-classification            0.731967
decision-tree-bow-classification    0.722131
naive-bayes-bow-classification      0.638525
naive-bayes-tfidf-classification    0.599180
naive-bayes-freq-classification     0.591803

Task: type.
Friedman chi square test p-value: 2.5197389103373674e-29
                                             Score
Actual                                    1.000000
svm-bow-type_of_antisemitism              0.582353
decision-tree-bow-type_of_antisemitism    0.576471
decision-tree-freq-type_of_antisemitism   0.567647
decision-tree-tfidf-type_of_antisemitism  0.555882
naive-bayes-freq-type_of_antisemitism     0.538235
naive-ba

In [44]:
for task in tasks:
    df = pd.read_csv(f'{task}_predictions.csv')
    for grouping in [representations, models]:
        temp_df = evaluate_models(df, grouping).transpose()
        temp_df = temp_df.sort_values('Score', ascending=False)
        friedman_result = test_model_difference(df, grouping)
        temp_df.columns = ['Score']
        
        print(f'Task: {task}. Grouping: {grouping[:3]}.')
        print(f'Friedman chi square test p-value: {friedman_result.pvalue}')
        print(temp_df)
        print(f'Mean: {temp_df["Score"].mean()}')
        print()

Task: binary. Grouping: ['bow', 'freq', 'tfidf'].
Friedman chi square test p-value: 1.1861381826157842e-16
          Score
bow    0.765574
freq   0.760656
tfidf  0.749180
Mean: 0.7584699453551913

Task: binary. Grouping: ['decision-tree', 'svm', 'naive-bayes'].
Friedman chi square test p-value: 1.985466342867531e-59
                  Score
decision-tree  0.767213
svm            0.754918
naive-bayes    0.600000
Mean: 0.707377049180328

Task: type. Grouping: ['bow', 'freq', 'tfidf'].
Friedman chi square test p-value: 0.0014689317923135177
          Score
freq   0.588235
bow    0.585294
tfidf  0.541176
Mean: 0.5715686274509805

Task: type. Grouping: ['decision-tree', 'svm', 'naive-bayes'].
Friedman chi square test p-value: 1.0019394785818424e-11
                  Score
decision-tree  0.582353
naive-bayes    0.535294
svm            0.523529
Mean: 0.5470588235294117



It can be concluded that both the model and text representation have significant results on the predictions, especially the model.